In [ ]:
import polars as pl
from huggingface_hub import snapshot_download
from datasets import load_dataset
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2


In [6]:
import os

from synthetic_data.tasks.writing import _get_gutenberg_subset

gutenberg_pq: pl.DataFrame = pl.read_parquet("../dataset_files/epubs.parquet")

In [ ]:
gutenberg_pq.shape

In [ ]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from IPython.display import Markdown
from synthetic_data.tasks.writing import GutenbergBacktranslationFromTxt

tiktoken_encoder = tiktoken.get_encoding("o200k_base")

first_sample = gutenberg_pq.sample(n=1)
task = GutenbergBacktranslationFromTxt()
total = 0

for row in gutenberg_pq.sample(10).iter_rows(named=True):
    print(row.keys())
    display(Markdown(f"**{row['title']}**"))
    row_out = await task.preprocess_row(row)
    for row in row_out:
        n_tokens = sum([len(tiktoken_encoder.encode(par)) for par in row['text']])
        # print("-"*100)
        # display(Markdown(row['text']))
        # print(f"n_tokens: {n_tokens}")
        total += 1
    print(total)

print(total)


In [ ]:
import re
from synthetic_data.utils import ldictl

input = first_sample.to_dicts()
formatted_out = task.format_input_conversation(ldictl(input))

In [ ]:
formatted_out